# L5: Information extraction

Information extraction (IE) is the task of identifying named entities and semantic relations between these entities in text data. In this lab we will focus on two sub-tasks in IE, **named entity recognition** (identifying mentions of entities) and **entity linking** (matching these mentions to entities in a knowledge base).

We start by loading spaCy:

In [1]:
import spacy

nlp = spacy.load('en_core_web_sm')

The data that we will be using has been tokenized following the conventions of the [Penn Treebank](ftp://ftp.cis.upenn.edu/pub/treebank/public_html/tokenization.html), and we need to prevent spaCy from using its own tokenizer on top of this. We therefore override spaCy&rsquo;s tokenizer with one that simply splits on space.

In [2]:
from spacy.tokens import Doc

class WhitespaceTokenizer(object):
    def __init__(self, vocab):
        self.vocab = vocab

    def __call__(self, text):
        return Doc(self.vocab, words=text.split(' '))

nlp.tokenizer = WhitespaceTokenizer(nlp.vocab)

## Data set

The main data set for this lab is a collection of news wire articles in which mentions of named entities have been annotated with page names from the [English Wikipedia](https://en.wikipedia.org/wiki/). The next code cell loads the training and the development parts of the data into Pandas data frames.

In [3]:
import bz2
import csv
import pandas as pd

with bz2.open('ner-train.tsv.bz2', 'rt') as source:
    df_train = pd.read_csv(source, sep='\t', quoting=csv.QUOTE_NONE)

with bz2.open('ner-dev.tsv.bz2', 'rt') as source:
    df_dev = pd.read_csv(source, sep='\t', quoting=csv.QUOTE_NONE)

Each row in these two data frames corresponds to one mention of a named entity and has five columns:

1. a unique identifier for the sentence containing the entity mention
2. the pre-tokenized sentence, with tokens separated by spaces
3. the start position of the token span containing the entity mention
4. the end position of the token span (exclusive, as in Python list indexing)
5. the entity label; either a Wikipedia page name or the generic label `--NME--`

The following cell prints the first five samples from the training data:

In [4]:
df_train.head()

,sentence_id,sentence,beg,end,label
0,0000-000,EU rejects German call to boycott British lamb .,0,1,--NME--
1,0000-000,EU rejects German call to boycott British lamb .,2,3,Germany
2,0000-000,EU rejects German call to boycott British lamb .,6,7,United_Kingdom
3,0000-001,Peter Blackburn,0,2,--NME--
4,0000-002,BRUSSELS 1996-08-22,0,1,Brussels


In this sample, we see that the first sentence is annotated with three entity mentions:

* the span 0–1 &lsquo;EU&rsquo; is annotated as a mention but only labelled with the generic `--NME--`
* the span 2–3 &lsquo;German&rsquo; is annotated with the page [Germany](http://en.wikipedia.org/wiki/Germany)
* the span 6–7 &lsquo;British&rsquo; is annotated with the page [United_Kingdom](http://en.wikipedia.org/wiki/United_Kingdom)

## Problem 1: Evaluation measures

To warm up, we ask you to write code to print the three measures that you will be using for evaluation:

In [5]:
def evaluation_report(gold, pred):
    """Print precision, recall, and F1 score.
    
    Args:
        gold: The set with the gold-standard values.
        pred: The set with the predicted values.
    
    Returns:
        Nothing, but prints the precision, recall, and F1 values computed
        based on the specified sets.
    """
    # TODO: Replace the next line with your own code
    gold = list(gold)
    pred = list(pred)
    
    precision_counter = 0
    recall_counter = 0
    for i, value in enumerate(gold):
      if gold[i] in pred:
        precision_counter+=1

    for i, value in enumerate(pred):
      if pred[i] in gold:
        recall_counter+=1

    precision = precision_counter/len(pred) 
    recall = recall_counter/len(gold)
    f1_score = 2 * precision * recall / (precision + recall)
    f1_score = round(f1_score, 2)

    print(f'The precision is: {precision} \n The recall is: {recall} \n The F1-value is: {f1_score}')

To test your code, you can run the following cell:

In [6]:
evaluation_report(set(range(3)), set(range(5)))

The precision is: 0.6 
 The recall is: 1.0 
 The F1-value is: 0.75


This should give you a precision of 60%, a recall of 100%, and an F1-value of 75%.

## Problem 2: Span recognition

One of the first tasks that an information extraction system has to solve is to locate and classify (mentions of) named entities, such as persons and organizations. Here we will tackle the simpler task of recognizing **spans** of tokens that contain an entity mention, without the actual entity label.

The English language model in spaCy features a full-fledged [named entity recognizer](https://spacy.io/usage/linguistic-features#named-entities) that identifies a variety of entities, and can be updated with new entity types by the user. Your task in this problem is to evaluate the performance of this component when predicting entity spans in the development data.

Start by implementing a generator function that yields the gold-standard spans in a given data frame.

**Hint:** The Pandas method [`itertuples()`](https://pandas.pydata.org/pandas-docs/version/0.17.0/generated/pandas.DataFrame.itertuples.html) is useful when iterating over the rows in a DataFrame.

In [7]:
df_dev.head(10)

,sentence_id,sentence,beg,end,label
0,0946-000,CRICKET - LEICESTERSHIRE TAKE OVER AT TOP AFTE...,2,3,Leicestershire_County_Cricket_Club
1,0946-001,LONDON 1996-08-30,0,1,London
2,0946-002,West Indian all-rounder Phil Simmons took four...,0,2,West_Indies_cricket_team
3,0946-002,West Indian all-rounder Phil Simmons took four...,3,5,Phil_Simmons
4,0946-002,West Indian all-rounder Phil Simmons took four...,12,13,Leicestershire_County_Cricket_Club
5,0946-002,West Indian all-rounder Phil Simmons took four...,14,15,Somerset_County_Cricket_Club
6,0946-003,"Their stay on top , though , may be short-live...",13,14,Essex_County_Cricket_Club
7,0946-003,"Their stay on top , though , may be short-live...",15,16,Derbyshire_County_Cricket_Club
8,0946-003,"Their stay on top , though , may be short-live...",17,18,Surrey_County_Cricket_Club
9,0946-003,"Their stay on top , though , may be short-live...",24,25,Kent_County_Cricket_Club


In [8]:
def gold_spans(df):
    """Yield the gold-standard mention spans in a data frame.

    Args:
        df: A data frame.

    Yields:
        The gold-standard mention spans in the specified data frame as
        triples consisting of the sentence id, start position, and end
        position of each span.
    """
    # TODO: Replace the next line with your own code
    for row in df.itertuples():
        yield row[1], row[3], row[4] # yield tuples
#     yield None

In [9]:
span_test = gold_spans(df_dev)
len(list(span_test)) # Unique triples


5917

In [10]:
for i in gold_spans(df_dev):
    print(i)

Streaming output truncated to the last 5000 lines.
('0966-131', 4, 5)
('0966-132', 1, 3)
('0966-132', 4, 5)
('0966-134', 1, 3)
('0966-134', 4, 5)
('0966-135', 1, 3)
('0966-135', 4, 5)
('0966-136', 1, 3)
('0966-136', 4, 5)
('0966-137', 1, 4)
('0966-137', 5, 6)
('0966-138', 1, 3)
('0966-138', 4, 5)
('0966-139', 1, 3)
('0966-139', 4, 5)
('0966-140', 1, 3)
('0966-140', 4, 5)
('0966-141', 1, 3)
('0966-141', 4, 5)
('0966-143', 1, 3)
('0966-143', 4, 5)
('0966-144', 1, 3)
('0966-144', 4, 5)
('0966-145', 1, 3)
('0966-145', 4, 5)
('0966-146', 1, 3)
('0966-146', 4, 5)
('0966-147', 1, 3)
('0966-147', 4, 5)
('0966-148', 1, 3)
('0966-148', 4, 5)
('0966-149', 1, 3)
('0966-149', 4, 5)
('0966-150', 1, 3)
('0966-150', 4, 5)
('0966-152', 1, 3)
('0966-152', 4, 5)
('0966-153', 1, 3)
('0966-153', 4, 5)
('0966-154', 1, 3)
('0966-154', 4, 5)
('0966-155', 1, 3)
('0966-155', 4, 5)
('0966-156', 1, 3)
('0966-156', 4, 5)
('0966-157', 1, 3)
('0966-157', 4, 5)
('0966-158', 1, 3)
('0966-158', 4, 5)
('0966-159', 1, 3)

To test your code, you can count the spans yielded by your function. When called on the development data, you should get a total of 5,917 unique triples. The first triple and the last triple should be

    ('0946-000', 2, 3)
    ('1161-010', 1, 3)  

In [11]:
spans_dev_gold = set(gold_spans(df_dev))
print(len(spans_dev_gold))

5917


Your next task is to write code that calls spaCy to predict the named entities in the development data, and to evaluate the accuracy of these predictions in terms of precision, recall, and F1. Print these scores using the function that you wrote for Problem&nbsp;1.

In [12]:
# TODO: Write code here to run and evaluate the spaCy NER on the development data

def spacy_entity(df):
    """Yield the gold-standard mention spans in a data frame.

    Args:
        df: A data frame.

    Yields:
        triples consisting of the sentence id, start position, and end
        position of each span.
    """
    # TODO: Replace the next line with your own code
    for row in df.itertuples():
        doc = nlp(row[2])
        for ent in doc.ents:
            yield row[1], ent.start, ent.end


In [13]:
dev_spacy_pred = set(spacy_entity(df_dev))

In [14]:
evaluation_report(spans_dev_gold, dev_spacy_pred)

The precision is: 0.529449922158796 
 The recall is: 0.6897076221057968 
 The F1-value is: 0.6


In [15]:
spans_dev_gold

{('0990-006', 9, 10),
 ('1025-000', 2, 3),
 ('1094-048', 0, 2),
 ('0968-003', 1, 5),
 ('0991-001', 0, 1),
 ('0992-002', 0, 1),
 ('1068-005', 0, 1),
 ('1084-003', 7, 9),
 ('1056-016', 25, 27),
 ('1054-019', 1, 3),
 ('0981-003', 2, 6),
 ('0990-014', 22, 23),
 ('1152-003', 29, 30),
 ('0956-010', 4, 5),
 ('0948-003', 4, 5),
 ('1011-016', 6, 7),
 ('1096-031', 26, 29),
 ('1155-014', 17, 18),
 ('1090-007', 33, 35),
 ('0997-008', 0, 1),
 ('1059-006', 18, 19),
 ('0998-003', 0, 1),
 ('0984-017', 0, 1),
 ('0977-007', 0, 1),
 ('1058-025', 0, 2),
 ('1039-015', 9, 13),
 ('1090-004', 0, 1),
 ('0966-021', 1, 3),
 ('0995-004', 9, 11),
 ('0957-006', 2, 4),
 ('0968-002', 0, 1),
 ('0953-003', 0, 1),
 ('0991-002', 0, 1),
 ('1090-014', 10, 12),
 ('0957-017', 10, 11),
 ('0963-024', 3, 5),
 ('1022-018', 1, 3),
 ('1158-003', 10, 11),
 ('1033-003', 19, 20),
 ('0986-002', 16, 18),
 ('1076-041', 0, 4),
 ('0966-101', 4, 5),
 ('0946-001', 0, 1),
 ('1127-015', 0, 1),
 ('1025-000', 0, 1),
 ('1126-000', 2, 3),
 ('0966

## Problem 3: Error analysis

As you were able to see in Problem&nbsp;2, the span accuracy of the named entity recognizer is far from perfect. In particular, only slightly more than half of the predicted spans are correct according to the gold standard. Your next task is to analyse this result in more detail.

Here is a function that prints the false positives as well as the false negatives spans for a data frame, given a reference set of gold-standard spans and a candidate set of predicted spans.

In [16]:
from collections import defaultdict

def error_report(df, spans_gold, spans_pred):
    false_pos = defaultdict(list)
    for s, b, e in spans_pred - spans_gold:
        false_pos[s].append((b, e))
    false_neg = defaultdict(list)
    for s, b, e in spans_gold - spans_pred:
        false_neg[s].append((b, e))
    for row in df.drop_duplicates('sentence_id').itertuples():
        if row.sentence_id in false_pos or row.sentence_id in false_neg:
            print('Sentence:', row.sentence)
            for b, e in false_pos[row.sentence_id]:
                print('  FP:', ' '.join(row.sentence.split()[b:e]))
            for b, e in false_neg[row.sentence_id]:
                print('  FN:', ' '.join(row.sentence.split()[b:e]))

Use this function to inspect and analyse the errors that the automated prediction makes. Can you see any patterns? Base your analysis on the first 500 rows of the training data. Summarize your observations in a short text.

In [17]:
# TODO: Write code here to do your analysis
error_report(df_dev, spans_dev_gold, dev_spacy_pred)

Streaming output truncated to the last 5000 lines.
  FP: 2.3 million
  FP: 1982
Sentence: Ironically , Bernardin , who developed a progressive , much-praised programme to deal with pedophilia involving priests , became the highest-ranking church official ever accused of sexual improprieties when a former seminary student leveled charges in 1993 that he had been sexually abused by Bernardin during the 1970s .
  FP: 1993
  FP: the 1970s
Sentence: Bernardin prayed with him before he died from AIDS last year .
  FP: last year
  FN: AIDS
Sentence: Bernardin , who became archbishop of Cincinnati in 1972 , played an increasingly prominent role in the U.S. church as a moderate voice of compromise among liberals and conservatives in the ranks of the bishops .
  FP: 1972
Sentence: This month , he announced that he would oversee a series of conferences to find " common ground " among American Catholics .
  FP: This month
Sentence: Boatmen 's deal could spark more mergers .
  FP: Boatmen
  FN: Boa

*TODO: Write a short text that summarises the errors that you observed* <br>

**We need to remove CARDINAL , DATE, TIME and ORDINAL from the Entity**

Now, use the insights from your error analysis to improve the automated prediction that you implemented in Problem&nbsp;2. While the best way to do this would be to [update spaCy&rsquo;s NER model](https://spacy.io/usage/linguistic-features#updating) using domain-specific training data, for this lab it suffices to write code to post-process the output produced by spaCy. You should be able to improve the F1 score from Problem&nbsp;2 by at last 15 percentage points.

In [27]:
# TODO: Write code here to improve the span prediction from Problem 2
def spacy_entity_improved(df):
    """Yield the gold-standard mention spans in a data frame.

    Args:
        df: A data frame.

    Yields:
        triples consisting of the sentence id, start position, and end
        position of each span.
    """
    # TODO: Replace the next line with your own code
    for row in df.itertuples():
        doc = nlp(row[2])
        for ent in doc.ents:
            if ent.label_ != 'CARDINAL' and ent.label_ != 'DATE' and ent.label_ != 'TIME' and ent.label_ != 'ORDINAL' and ent.label_ != 'PERCENT':
                yield row[1], ent.start, ent.end

In [28]:
dev_spacy_pred_new = set(spacy_entity_improved(df_dev))

In [29]:
evaluation_report(spans_dev_gold, dev_spacy_pred_new)

The precision is: 0.819955743311205 
 The recall is: 0.6888625992901808 
 The F1-value is: 0.75


In [30]:
error_report(df_dev, spans_dev_gold, dev_spacy_pred_new)

Sentence: CRICKET - LEICESTERSHIRE TAKE OVER AT TOP AFTER INNINGS VICTORY .
  FN: LEICESTERSHIRE
Sentence: West Indian all-rounder Phil Simmons took four for 38 on Friday as Leicestershire beat Somerset by an innings and 39 runs in two days to take over at the head of the county championship .
  FN: Leicestershire
Sentence: Trailing by 213 , Somerset got a solid start to their second innings before Simmons stepped in to bundle them out for 174 .
  FN: Simmons
Sentence: Essex , however , look certain to regain their top spot after Nasser Hussain and Peter Such gave them a firm grip on their match against Yorkshire at Headingley .
  FP: Yorkshire at Headingley
  FN: Essex
  FN: Headingley
  FN: Yorkshire
Sentence: Hussain , considered surplus to England 's one-day requirements , struck 158 , his first championship century of the season , as Essex reached 372 and took a first innings lead of 82 .
  FN: Hussain
Sentence: By the close Yorkshire had turned that into a 37-run advantage but of

Show that you achieve the performance goal by reporting the evaluation measures that you implemented in Problem&nbsp;1.

Before going on, we ask you to store the outputs of the improved named entity recognizer on the development data in a new data frame. This new frame should have the same layout as the original data frame for the development data that you loaded above, but should contain the *predicted* start and end positions for each token span, rather than the gold positions. As the `label` of each span, you can use the special value `--NME--`.

In [33]:
# TODO: Write code here to store the predicted spans in a new data frame
import numpy as np

def generate_new_df(df):
    """Yield the gold-standard mention spans in a data frame.

    Args:
        df: A data frame.

    return:
        panda DF with ID, Sentence, token start, token end, and label
    """
    id_list = []
    sentence_list = []
    start_list = []
    end_list = []
    label_list = []
    
    for row in df.itertuples():
        doc = nlp(row[2])
        if row[1] not in id_list:
            for ent in doc.ents:
                if ent.label_ != 'CARDINAL' and ent.label_ != 'DATE' and ent.label_ != 'TIME' and ent.label_ != 'ORDINAL' and ent.label_ != 'PERCENT':
                    id_list.append(row[1])
                    sentence_list.append(row[2])
                    start_list.append(ent.start)
                    end_list.append(ent.end)
                    label_list.append('--NME--')
    return pd.DataFrame(data={'sentence_id': id_list, 
                              'sentence': sentence_list,
                              'beg': start_list,
                              'end': end_list,
                              'label': label_list})


In [34]:
new_df_dev = generate_new_df(df_dev)

In [35]:
new_df_dev.head(20)

,sentence_id,sentence,beg,end,label
0,0946-001,LONDON 1996-08-30,0,1,--NME--
1,0946-002,West Indian all-rounder Phil Simmons took four...,0,2,--NME--
2,0946-002,West Indian all-rounder Phil Simmons took four...,3,5,--NME--
3,0946-002,West Indian all-rounder Phil Simmons took four...,14,15,--NME--
4,0946-003,"Their stay on top , though , may be short-live...",13,14,--NME--
5,0946-003,"Their stay on top , though , may be short-live...",15,16,--NME--
6,0946-003,"Their stay on top , though , may be short-live...",17,18,--NME--
7,0946-003,"Their stay on top , though , may be short-live...",24,25,--NME--
8,0946-003,"Their stay on top , though , may be short-live...",35,36,--NME--
9,0946-004,After bowling Somerset out for 83 on the openi...,2,3,--NME--


## Problem 4: Entity linking

Now that we have a method for predicting mention spans, we turn to the task of **entity linking**, which amounts to predicting the knowledge base entity that is referenced by a given mention. In our case, for each span we want to predict the Wikipedia page that this mention references.

Start by extending the generator function that you implemented in Problem&nbsp;2 to labelled spans.

In [36]:
def gold_mentions(df):
    """Yield the gold-standard mentions in a data frame.

    Args:
        df: A data frame.

    Yields:
        The gold-standard mention spans in the specified data frame as
        quadruples consisting of the sentence id, start position, end
        position and entity label of each span.
    """
    # TODO: Replace the next line with your own code
    for row in df.itertuples():
        yield row[1], row[3], row[4], row[5]  # yield quadruples
    yield None

In [37]:
spans_dev_gold_mentions = set(gold_mentions(df_dev))

In [38]:
spans_dev_gold_mentions

{('0972-006', 0, 2, '--NME--'),
 ('1032-007', 6, 7, 'Germany'),
 ('0965-007', 19, 20, 'Donovan_Bailey'),
 ('1066-071', 0, 1, 'Chester_City_F.C.'),
 ('1123-010', 0, 1, '--NME--'),
 ('1115-010', 14, 16, 'My_Turn_(memoir)'),
 ('1108-002', 22, 24, 'James_Heath_(boxer)'),
 ('0974-010', 0, 2, '--NME--'),
 ('1150-006', 13, 16, '--NME--'),
 ('0947-009', 18, 19, 'Lancashire_County_Cricket_Club'),
 ('1099-017', 11, 13, 'Anton_Pfeffer'),
 ('0966-110', 4, 5, 'Germany'),
 ('1008-027', 1, 4, 'University_of_Michigan'),
 ('1076-017', 0, 1, '--NME--'),
 ('0948-029', 9, 10, 'Birmingham'),
 ('0952-000', 2, 3, 'FC_Rotor_Volgograd'),
 ('1035-009', 0, 1, 'Russia'),
 ('0997-004', 27, 28, 'Moscow'),
 ('1150-006', 2, 3, 'Boston'),
 ('0991-001', 0, 1, 'Khasavyurt'),
 ('1155-010', 0, 1, 'Abolhassan_Banisadr'),
 ('0962-008', 7, 9, 'Jesper_Parnevik'),
 ('0946-014', 9, 11, 'Paul_Johnson_(cricketer)'),
 ('1096-030', 0, 1, 'Montreal_Expos'),
 ('1046-002', 28, 29, 'Spain'),
 ('1055-007', 4, 5, 'Ata-ur-Rahman'),
 ('107

A naive baseline for entity linking on our data set is to link each mention span to the Wikipedia page name that we get when we join the tokens in the span by underscores, as is standard in Wikipedia page names. Suppose, for example, that a span contains the two tokens

    Jimi Hendrix

The baseline Wikipedia page name for this span would be

    Jimi_Hendrix

Implement this naive baseline and evaluate its performance. Print the evaluation measures that you implemented in Problem&nbsp;1.

<div class="alert alert-warning">
    Here and in the remainder of this lab, you should base your entity predictions on the predicted spans that you computed in Problem&nbsp;3.
</div>

In [39]:
# TODO: Write code here to implement the baseline
def naive_pred(df):
    """Yield the gold-standard mentions in a data frame.

    Args:
        df: A data frame.

    Yields:
        The gold-standard mention spans in the specified data frame as
        quadruples consisting of the sentence id, start position, end
        position and entity label of each span.
    """
    # TODO: Replace the next line with your own code
    for row in df.itertuples():
        new_t = row[2]
        new_t = new_t.split(' ')
        yield row[1], row[3], row[4], "_".join(new_t[row[3]:row[4]])  # yield quadruples
    yield None

In [40]:
dev_naive_pred_new = set(naive_pred(new_df_dev))

In [41]:
dev_naive_pred_new

{('1116-012', 5, 7, 'Henry_Campbell-Bannerman'),
 ('1119-007', 9, 10, 'British'),
 ('1099-018', 2, 4, 'Andrew_Goram'),
 ('1030-016', 7, 8, 'Treasury'),
 ('1119-005', 11, 12, 'Iraqi'),
 ('0992-003', 1, 2, 'Bosnian'),
 ('1096-042', 8, 9, 'RBI'),
 ('1051-009', 32, 34, 'Klas_Eriksson'),
 ('1057-006', 2, 4, 'Mike_Atherton'),
 ('1068-024', 0, 2, 'Notts_County'),
 ('0959-005', 2, 4, 'Kansas_City'),
 ('1019-009', 0, 1, 'German'),
 ('1069-008', 4, 6, 'Durham_4'),
 ('1051-010', 18, 19, 'Australia'),
 ('1035-009', 15, 16, 'Germany'),
 ('0957-020', 18, 20, 'Czech_Republic'),
 ('1022-003', 23, 25, 'Jim_Webber'),
 ('0983-002', 35, 36, 'Britain'),
 ('0961-004', 6, 7, 'Cunningham'),
 ('1058-037', 0, 2, 'Van_Poppel'),
 ('1138-007', 3, 4, 'Wang'),
 ('1055-028', 1, 2, 'Akram'),
 ('0981-005', 26, 27, '31'),
 ('1103-016', 42, 43, 'Kafkas'),
 ('1044-003', 13, 14, 'Authority'),
 ('1116-029', 10, 11, 'Italian'),
 ('1077-003', 16, 17, 'England'),
 ('1124-006', 0, 1, 'Viljoen'),
 ('1005-014', 19, 25, 'between_$

In [42]:
evaluation_report(spans_dev_gold_mentions, dev_naive_pred_new)

The precision is: 0.30913113435237327 
 The recall is: 0.25971612031091584 
 The F1-value is: 0.28


## Problem 5: Extending the training data using the knowledge base

State-of-the-art approaches to entity linking exploit information in knowledge bases. In our case, where Wikipedia is the knowledge base, one particularly useful type of information are links to other Wikipedia pages. In particular, we can interpret the anchor texts (the highlighted texts that you click on) as mentions of the entities (pages) that they link to. This allows us to harvest long lists of mention–entity pairings.

The following cell loads a data frame summarizing anchor texts and page references harvested from the first paragraphs of the English Wikipedia. The data frame also contains all entity mentions in the training data (but not the development or the test data).

In [43]:
with bz2.open('kb.tsv.bz2', 'rt') as source:
    df_kb = pd.read_csv(source, sep='\t', quoting=csv.QUOTE_NONE)

To understand what information is available in this data, the following cell shows the entry for the anchor text `Sweden`.

In [44]:
df_kb.loc[df_kb.mention == 'Sweden']

,mention,entity,prob
17436,Sweden,Sweden,0.985768
17437,Sweden,Sweden_national_football_team,0.014173
17438,Sweden,Sweden_men's_national_ice_hockey_team,0.000059


As you can see, each row of the data frame contains a pair $(m, e)$ of a mention $m$ and an entity $e$, as well as the conditional probability $P(e|m)$ for mention $m$ referring to entity $e$. These probabilities were estimated based on the frequencies of mention–entity pairs in the knowledge base. The example shows that the anchor text &lsquo;Sweden&rsquo; is most often used to refer to the entity [Sweden](http://en.wikipedia.org/wiki/Sweden), but in a few cases also to refer to Sweden&rsquo;s national football and ice hockey teams. Note that references are sorted in decreasing order of probability, so that the most probable pairing come first.

Implement an entity linking method that resolves each mention to the most probable entity in the data frame. If the mention is not included in the data frame, you can predict the generic label `--NME--`. Print the precision, recall, and F1 of your method using the function that you implemented for Problem&nbsp;1.

In [45]:
# TODO: Write code here to implement the "most probable entity" method.
def mark_labels(df):
    """Yield the gold-standard mentions in a data frame.

    Args:
        df: A data frame.

    return:
        Return DF with label - Most Probable Entity Link
    """
    id_list = []
    sentence_list = []
    start_list = []
    end_list = []
    label_list = []
    
    for row in df.itertuples():
        sent_text = row[2]
        sent_text = sent_text.split(' ')
        
        id_list.append(row[1])
        sentence_list.append(row[2])
        start_list.append(row[3])
        end_list.append(row[4])
        if len(df_kb.loc[df_kb.mention == " ".join(sent_text[row[3]:row[4]])]) != 0:
            entity_data = df_kb.loc[df_kb.mention == " ".join(sent_text[row[3]:row[4]])]
            label_list.append(entity_data['entity'].tolist()[0])
        else:
            label_list.append('--NME--')
    return pd.DataFrame(data={'sentence_id': id_list, 
                              'sentence': sentence_list,
                              'beg': start_list,
                              'end': end_list,
                              'label': label_list})

In [46]:
new_labeled_df_dev = mark_labels(new_df_dev)

In [47]:
new_labeled_df_dev.head(20)

,sentence_id,sentence,beg,end,label
0,0946-001,LONDON 1996-08-30,0,1,London
1,0946-002,West Indian all-rounder Phil Simmons took four...,0,2,West_Indies_cricket_team
2,0946-002,West Indian all-rounder Phil Simmons took four...,3,5,Phil_Simmons
3,0946-002,West Indian all-rounder Phil Simmons took four...,14,15,Somerset_County_Cricket_Club
4,0946-003,"Their stay on top , though , may be short-live...",13,14,Essex
5,0946-003,"Their stay on top , though , may be short-live...",15,16,Derbyshire_County_Cricket_Club
6,0946-003,"Their stay on top , though , may be short-live...",17,18,Surrey
7,0946-003,"Their stay on top , though , may be short-live...",24,25,Kent_County_Cricket_Club
8,0946-003,"Their stay on top , though , may be short-live...",35,36,Nottinghamshire_County_Cricket_Club
9,0946-004,After bowling Somerset out for 83 on the openi...,2,3,Somerset_County_Cricket_Club


In [48]:
df_dev_pred_mentions = set(gold_mentions(new_labeled_df_dev))

In [49]:
df_dev_pred_mentions

{('0972-006', 0, 2, '--NME--'),
 ('1128-003', 19, 21, '--NME--'),
 ('0965-007', 19, 20, 'Donovan_Bailey'),
 ('1132-002', 15, 16, '--NME--'),
 ('1123-010', 0, 1, '--NME--'),
 ('0957-033', 12, 13, '--NME--'),
 ('1051-009', 32, 34, 'Klas_Eriksson'),
 ('0966-051', 4, 5, 'United_Kingdom'),
 ('1065-021', 0, 1, 'Hamilton,_New_Zealand'),
 ('1118-003', 11, 12, 'Labour_Party_(UK)'),
 ('1072-017', 0, 3, '--NME--'),
 ('1051-010', 18, 19, 'Australia'),
 ('1035-009', 15, 16, 'Germany'),
 ('0966-007', 1, 3, '--NME--'),
 ('0957-020', 18, 20, 'Czech_Republic'),
 ('0974-010', 0, 2, '--NME--'),
 ('1142-013', 6, 7, 'Kurds'),
 ('1072-015', 7, 9, '--NME--'),
 ('1077-003', 16, 17, 'England'),
 ('1140-002', 11, 13, 'Hong_Kong'),
 ('0966-110', 4, 5, 'Germany'),
 ('0966-118', 4, 5, 'Russia'),
 ('0947-010', 20, 21, 'Hampshire'),
 ('0972-011', 0, 2, '--NME--'),
 ('1035-010', 14, 22, '--NME--'),
 ('0948-029', 9, 10, 'Birmingham'),
 ('0966-054', 1, 3, 'Lars_Riedel'),
 ('0982-000', 0, 1, 'United_Kingdom'),
 ('1055-0

In [50]:
evaluation_report(spans_dev_gold_mentions, df_dev_pred_mentions)

The precision is: 0.6309332260659695 
 The recall is: 0.5300777289624873 
 The F1-value is: 0.58


## Problem 6: Context-sensitive disambiguation

Consider the entity mention &lsquo;Lincoln&rsquo;. The most probable entity for this mention turns out to be [Lincoln, Nebraska](http://en.wikipedia.org/Lincoln,_Nebraska); but in pages about American history, we would be better off to predict [Abraham Lincoln](http://en.wikipedia.org/Abraham_Lincoln). This suggests that we should try to disambiguate between different entity references based on the textual context on the page from which the mention was taken. Your task in this last problem is to implement this idea.

Set up a dictionary that contains, for each mention $m$ that can refer to more than one entity $e$, a separate Naive Bayes classifier that is trained to predict the correct entity $e$, given the textual context of the mention. As the prior probabilities of the classifier, choose the probabilities $P(e|m)$ that you used in Problem&nbsp;5. To let you estimate the context-specific probabilities, we have compiled a data set with mention contexts:

In [51]:
with bz2.open('contexts.tsv.bz2') as source:
    df_contexts = pd.read_csv(source, sep='\t', quoting=csv.QUOTE_NONE)

This data frame contains, for each ambiguous mention $m$ and each knowledge base entity $e$ to which this mention can refer, up to 100 randomly selected contexts in which $m$ is used to refer to $e$. For this data, a **context** is defined as the 5 tokens to the left and the 5 tokens to the right of the mention. Here are a few examples:

In [52]:
df_contexts.head()

,mention,entity,context
0,1970,UEFA_Champions_League,Cup twice the first in @ and the second in 1983
1,1970,FIFA_World_Cup,America 1975 and during the @ and 1978 World C...
2,1990 World Cup,1990_FIFA_World_Cup,Manolo represented Spain at the @
3,1990 World Cup,1990_FIFA_World_Cup,Hašek represented Czechoslovakia at the @ and ...
4,1990 World Cup,1990_FIFA_World_Cup,renovations in 1989 for the @ The present capa...


Note that, in each context, the position of the mention is indicated by the `@` symbol.

From this data frame, it is easy to select the data that you need to train the classifiers – the contexts and corresponding entities for all mentions. To illustrate this, the following cell shows how to select all contexts that belong to the mention &lsquo;Lincoln&rsquo;:

In [53]:
df_contexts.context[df_contexts.mention == 'Lincoln']

41465    Nebraska Concealed Handgun Permit In @ municip...
41466    Lazlo restaurants are located in @ and Omaha C...
41467    California Washington Overland Park Kansas @ N...
41468    City Missouri Omaha Nebraska and @ Nebraska It...
41469    by Sandhills Publishing Company in @ Nebraska USA
                               ...                        
41609                                      @ Leyton Orient
41610                    English division three Swansea @ 
41611    league membership narrowly edging out @ on goa...
41612                                          @ Cambridge
41613                                                   @ 
Name: context, Length: 149, dtype: object

In [54]:
df_contexts[df_contexts.mention == 'Lincoln']

,mention,entity,context
41465,Lincoln,"Lincoln,_Nebraska",Nebraska Concealed Handgun Permit In @ municip...
41466,Lincoln,"Lincoln,_Nebraska",Lazlo restaurants are located in @ and Omaha C...
41467,Lincoln,"Lincoln,_Nebraska",California Washington Overland Park Kansas @ N...
41468,Lincoln,"Lincoln,_Nebraska",City Missouri Omaha Nebraska and @ Nebraska It...
41469,Lincoln,"Lincoln,_Nebraska",by Sandhills Publishing Company in @ Nebraska USA
...,...,...,...
41609,Lincoln,Lincoln_City_F.C.,@ Leyton Orient
41610,Lincoln,Lincoln_City_F.C.,English division three Swansea @
41611,Lincoln,Lincoln_City_F.C.,league membership narrowly edging out @ on goa...
41612,Lincoln,Lincoln_City_F.C.,@ Cambridge


Implement the context-sensitive disambiguation method and evaluate its performance. Here are some more hints that may help you along the way:

**Hint 1:** The prior probabilities for a Naive Bayes classifier can be specified using the `class_prior` option. You will have to provide the probabilities in the same order as the alphabetically sorted class (entity) names.

**Hint 2:** Not all mentions in the knowledge base are ambiguous, and therefore not all mentions have context data. If a mention has only one possible entity, pick that one. If a mention has no entity at all, predict the `--NME--` label.

In [55]:
# TODO: Write code here to implement the context-sensitive disambiguation method
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer

def mark_labels(df):
    """Yield the gold-standard mentions in a data frame.

    Args:
        df: A data frame.

    return:
        Return DF with label - Most Probable Entity Link
    """
    id_list = []
    sentence_list = []
    start_list = []
    end_list = []
    label_list = []
    
    for row in df.itertuples():
        men_text = row[5]
        men_text = men_text.split(' ')
        men_text = " ".join(men_text)
        
        id_list.append(row[1])
        sentence_list.append(row[2])
        start_list.append(row[3])
        end_list.append(row[4])
        
        mentioned_data = df_contexts[df_contexts.mention == men_text]
        
        if len(mentioned_data) != 0:
            if len(mentioned_data['entity'].tolist()) != 1:
                df_kb.loc[df_kb.mention == 'Sweden']
                entity_tags = df_kb.loc[df_kb.mention == row[5]]
                prior_prob = entity_tags.sort_values('entity', ascending=True)['prob'].tolist()
                
                pipe = Pipeline([('Tfid_Vect',TfidfVectorizer()),('MNB', MultinomialNB(class_prior=prior_prob))])
                pipe.fit(df_contexts[df_contexts.mention == men_text]['context'], 
                         df_contexts[df_contexts.mention == men_text]['entity'].tolist())
                y_pred = pipe.predict([row[2]])
                label_list.append(y_pred[0])
            
            else:
                label_list.append(mentioned_data['entity'].tolist()[0])
        else:
            label_list.append(row[5])
    return pd.DataFrame(data={'sentence_id': id_list, 
                              'sentence': sentence_list,
                              'beg': start_list,
                              'end': end_list,
                              'label': label_list})

In [56]:
df_dev_new_labeled = mark_labels(new_labeled_df_dev)

In [57]:
df_dev_new_labeled.head()

,sentence_id,sentence,beg,end,label
0,0946-001,LONDON 1996-08-30,0,1,London
1,0946-002,West Indian all-rounder Phil Simmons took four...,0,2,West_Indies_cricket_team
2,0946-002,West Indian all-rounder Phil Simmons took four...,3,5,Phil_Simmons
3,0946-002,West Indian all-rounder Phil Simmons took four...,14,15,Somerset_County_Cricket_Club
4,0946-003,"Their stay on top , though , may be short-live...",13,14,Essex_County_Cricket_Club


In [58]:
dev_pred_mentions = set(gold_mentions(df_dev_new_labeled))

In [59]:
dev_pred_mentions

{('0972-006', 0, 2, '--NME--'),
 ('1128-003', 19, 21, '--NME--'),
 ('0965-007', 19, 20, 'Donovan_Bailey'),
 ('1132-002', 15, 16, '--NME--'),
 ('1123-010', 0, 1, '--NME--'),
 ('0957-033', 12, 13, '--NME--'),
 ('1051-009', 32, 34, 'Klas_Eriksson'),
 ('0966-051', 4, 5, 'United_Kingdom'),
 ('1065-021', 0, 1, 'Hamilton,_New_Zealand'),
 ('1118-003', 11, 12, 'Labour_Party_(UK)'),
 ('1072-017', 0, 3, '--NME--'),
 ('1051-010', 18, 19, 'Australia'),
 ('1035-009', 15, 16, 'Germany'),
 ('0966-007', 1, 3, '--NME--'),
 ('0957-020', 18, 20, 'Czech_Republic'),
 ('0974-010', 0, 2, '--NME--'),
 ('1142-013', 6, 7, 'Kurds'),
 ('1072-015', 7, 9, '--NME--'),
 ('1077-003', 16, 17, 'England'),
 ('1140-002', 11, 13, 'Hong_Kong'),
 ('0966-110', 4, 5, 'Germany'),
 ('0966-118', 4, 5, 'Russia'),
 ('0947-010', 20, 21, 'Hampshire'),
 ('0972-011', 0, 2, '--NME--'),
 ('1035-010', 14, 22, '--NME--'),
 ('0948-029', 9, 10, 'Birmingham'),
 ('0966-054', 1, 3, 'Lars_Riedel'),
 ('0982-000', 0, 1, 'United_Kingdom'),
 ('1055-0

In [60]:
evaluation_report(spans_dev_gold_mentions, dev_pred_mentions)

The precision is: 0.6319388576025744 
 The recall is: 0.5309226089895235 
 The F1-value is: 0.58


You should expect to see a small (around 1&nbsp;unit) increase in both precision, recall, and F1.

**This was the last lab in the Text Mining course. Congratulations!**

<div class="alert alert-info">
    Please read the section ‘General information’ on the ‘Labs’ page of the course website before submitting this notebook!
</div>